<img align="right" src="images/tf-small.png" width="128"/>
<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/dans-small.png"/>

You might want to consider the [start](search.ipynb) of this tutorial.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tf.app import use

In [3]:
VERSION = '2017'

In [4]:
A = use('bhsa', hoist=globals(), version=VERSION)

Using etcbc/bhsa/tf - 2017 r1.4 in /Users/dirk/text-fabric-data
Using etcbc/phono/tf - 2017 r1.1 in /Users/dirk/text-fabric-data
Using etcbc/parallels/tf - 2017 r1.1 in /Users/dirk/text-fabric-data


**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="provenance of BHSA = Biblia Hebraica Stuttgartensia Amstelodamensis">BHSA</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Writing/Hebrew" title="('Hebrew characters and transcriptions',)">Character table</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/2017/0_home.html" title="BHSA feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/Bhsa/" title="bhsa API documentation">bhsa API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 7.1.0</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Search/#search-templates" title="Search Templates Introduction and Reference">Search Reference</a>

# Gaps and spans

Searches often do not deliver the results you expect.
Besides typos, lack of familiarity with the template formalism and bugs in the system, there is
another cause: **difficult semantics of the data**.

Most users reason about phrases, clauses and sentences as if they are consecutive blocks of words.
But in the BHSA this is not the case: each of these objects may have **gaps**.

Most of the time, verse boundaries coincide with the boundaries of sentences, clauses, and phrases.
But not always, there are verse **spanning** sentences.

> **Note**
These phenomena may wreak havoc with your intuitive reasoning about what search templates should deliver.
Query templates do not require the objects to be consecutive and
still they make sense. But that might not be your sense, unless you **Mind the gap!**

We are going to show these issues in depth.

# Gaps

TF-search has no primitives to deal with gaps directly.
Nodes correspond to textual objects such as words, phrases, clauses, verses, books.
Usually these are consecutive sequences of one or more words,
but in theory they can be arbitrary sets of slots.

And, as far as the BHSA corpus is concerned, in practice too.
If we look at phrases, then the overwhelming majority is consecutive, without gaps,
But there is also a substantial amount of phrases with gaps.

People that are familiar with MQL (see [fromMQL](searchFromMQL.ipynb))
may remember that in MQL you can search for a gap.
The MQL query

```
SELECT ALL OBJECTS WHERE

[phrase FOCUS
    [word lex='L']
    [gap]
]
```

looks for a phrase with a gap in it
(i.e. one or more consecutive words between the start and the end of the phrase
that do not belong to the phrase).
The query then asks additionally for those gap-containing phrases that have a certain word in front of the gap.

**We want this too!**

## Find the gap

We start with a query that aims to get the same results as the MQL query above.

In our template, we require that there is a word `wPreGap` in the phrase that is just before the gap,
a word `wGap` that comes right after, so it is in the gap, and hence does not belong to the phrase.
But this all must happen before the last word `wLast` of the phrase.

In [5]:
query = '''
verse
    p:phrase
      wPreGap:word lex=L
      wLast:word
      :=

wGap:word
wPreGap <: wGap
wGap < wLast
p || wGap
'''

In [6]:
results = A.search(query)

  0.77s 13 results


Nice and quick.
Let's see the results.

In [7]:
A.table(results)

n | verse | phrase | word | word | word
--- | --- | --- | --- | --- | ---
1|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=17&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 17:7" sec="Genesis 17:7">Genesis 17:7</a>וַהֲקִמֹתִ֨י אֶת־בְּרִיתִ֜י בֵּינִ֣י וּבֵינֶ֗ךָ וּבֵ֨ין זַרְעֲךָ֧ אַחֲרֶ֛יךָ לְדֹרֹתָ֖ם לִבְרִ֣ית עֹולָ֑ם לִהְיֹ֤ות לְךָ֙ לֵֽאלֹהִ֔ים וּֽלְזַרְעֲךָ֖ אַחֲרֶֽיךָ׃ </span>|<span class="hb">לְךָ֙ וּֽלְזַרְעֲךָ֖ אַחֲרֶֽיךָ׃ </span>|<span class="hb">לְךָ֙ </span>|<span class="hb">אַחֲרֶֽיךָ׃ </span>|<span class="hb">לֵֽ</span>
2|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=28&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 28:4" sec="Genesis 28:4">Genesis 28:4</a>וְיִֽתֶּן־לְךָ֙ אֶת־בִּרְכַּ֣ת אַבְרָהָ֔ם לְךָ֖ וּלְזַרְעֲךָ֣ אִתָּ֑ךְ לְרִשְׁתְּךָ֙ אֶת־אֶ֣רֶץ מְגֻרֶ֔יךָ אֲשֶׁר־נָתַ֥ן אֱלֹהִ֖ים לְאַבְרָהָֽם׃ </span>|<span class="hb">לְךָ֙ לְךָ֖ וּלְזַרְעֲךָ֣ אִתָּ֑ךְ </span>|<span class="hb">לְךָ֙ </span>|<span class="hb">אִתָּ֑ךְ </span>|<span class="hb">אֶת־</span>
3|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 31:16" sec="Genesis 31:16">Genesis 31:16</a>כִּ֣י כָל־הָעֹ֗שֶׁר אֲשֶׁ֨ר הִצִּ֤יל אֱלֹהִים֙ מֵֽאָבִ֔ינוּ לָ֥נוּ ה֖וּא וּלְבָנֵ֑ינוּ וְעַתָּ֗ה כֹּל֩ אֲשֶׁ֨ר אָמַ֧ר אֱלֹהִ֛ים אֵלֶ֖יךָ עֲשֵֽׂה׃ </span>|<span class="hb">לָ֥נוּ וּלְבָנֵ֑ינוּ </span>|<span class="hb">לָ֥נוּ </span>|<span class="hb">בָנֵ֑ינוּ </span>|<span class="hb">ה֖וּא </span>
4|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 30:21" sec="Exodus 30:21">Exodus 30:21</a>וְרָחֲצ֛וּ יְדֵיהֶ֥ם וְרַגְלֵיהֶ֖ם וְלֹ֣א יָמֻ֑תוּ וְהָיְתָ֨ה לָהֶ֧ם חָק־עֹולָ֛ם לֹ֥ו וּלְזַרְעֹ֖ו לְדֹרֹתָֽם׃ פ </span>|<span class="hb">לָהֶ֧ם לֹ֥ו וּלְזַרְעֹ֖ו </span>|<span class="hb">לָהֶ֧ם </span>|<span class="hb">זַרְעֹ֖ו </span>|<span class="hb">חָק־</span>
5|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Leviticus&chapter=25&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Leviticus 25:6" sec="Leviticus 25:6">Leviticus 25:6</a>וְ֠הָיְתָה שַׁבַּ֨ת הָאָ֤רֶץ לָכֶם֙ לְאָכְלָ֔ה לְךָ֖ וּלְעַבְדְּךָ֣ וְלַאֲמָתֶ֑ךָ וְלִשְׂכִֽירְךָ֙ וּלְתֹושָׁ֣בְךָ֔ הַגָּרִ֖ים עִמָּֽךְ׃ </span>|<span class="hb">לָכֶם֙ לְךָ֖ וּלְעַבְדְּךָ֣ וְלַאֲמָתֶ֑ךָ וְלִשְׂכִֽירְךָ֙ וּלְתֹושָׁ֣בְךָ֔ </span>|<span class="hb">לָכֶם֙ </span>|<span class="hb">תֹושָׁ֣בְךָ֔ </span>|<span class="hb">לְ</span>
6|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=20&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 20:15" sec="Numbers 20:15">Numbers 20:15</a>וַיֵּרְד֤וּ אֲבֹתֵ֨ינוּ֙ מִצְרַ֔יְמָה וַנֵּ֥שֶׁב בְּמִצְרַ֖יִם יָמִ֣ים רַבִּ֑ים וַיָּרֵ֥עוּ לָ֛נוּ מִצְרַ֖יִם וְלַאֲבֹתֵֽינוּ׃ </span>|<span class="hb">לָ֛נוּ וְלַאֲבֹתֵֽינוּ׃ </span>|<span class="hb">לָ֛נוּ </span>|<span class="hb">אֲבֹתֵֽינוּ׃ </span>|<span class="hb">מִצְרַ֖יִם </span>
7|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=32&verse=33&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 32:33" sec="Numbers 32:33">Numbers 32:33</a>וַיִּתֵּ֣ן לָהֶ֣ם׀ מֹשֶׁ֡ה לִבְנֵי־גָד֩ וְלִבְנֵ֨י רְאוּבֵ֜ן וְלַחֲצִ֣י׀ שֵׁ֣בֶט׀ מְנַשֶּׁ֣ה בֶן־יֹוסֵ֗ף אֶת־מַמְלֶ֨כֶת֙ סִיחֹן֙ מֶ֣לֶךְ הָֽאֱמֹרִ֔י וְאֶת־מַמְלֶ֔כֶת עֹ֖וג מֶ֣לֶךְ הַבָּשָׁ֑ן הָאָ֗רֶץ לְעָרֶ֨יהָ֙ בִּגְבֻלֹ֔ת עָרֵ֥י הָאָ֖רֶץ סָבִֽיב׃ </span>|<span class="hb">לָהֶ֣ם׀ לִבְנֵי־גָד֩ וְלִבְנֵ֨י רְאוּבֵ֜ן וְלַחֲצִ֣י׀ שֵׁ֣בֶט׀ מְנַשֶּׁ֣ה בֶן־יֹוסֵ֗ף </span>|<span class="hb">לָהֶ֣ם׀ </span>|<span class="hb">יֹוסֵ֗ף </span>|<span class="hb">מֹשֶׁ֡ה </span>
8|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=1&verse=36&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 1:36" sec="Deuteronomy 1:36">Deuteronomy 1:36</a>זֽוּלָתִ֞י כָּלֵ֤ב בֶּן־יְפֻנֶּה֙ ה֣וּא יִרְאֶ֔נָּה וְלֹֽו־אֶתֵּ֧ן אֶת־הָאָ֛רֶץ אֲשֶׁ֥ר דָּֽרַךְ־בָּ֖הּ וּלְבָנָ֑יו יַ֕עַן אֲשֶׁ֥ר מִלֵּ֖א אַחֲרֵ֥י יְהוָֽה׃ </span>|<span class="hb">לֹֽו־וּלְבָנָ֑יו </span>|<span class="hb">לֹֽו־</span>|<span class="hb">בָנָ֑יו </span>|<span class="hb">אֶתֵּ֧ן </span>
9|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=26&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 26:11" sec="Deuteronomy 26:11">Deuteronomy 26:11</a>וְשָׂמַחְתָּ֣ בְכָל־הַטֹּ֗וב אֲשֶׁ֧ר נָֽתַן־לְךָ֛ יְהוָ֥ה אֱלֹהֶ֖יךָ וּלְבֵיתֶ֑ךָ אַתָּה֙ וְהַלֵּוִ֔י וְהַגֵּ֖ר אֲשֶׁ֥ר בְּקִרְבֶּֽךָ׃ ס </span>|<span class="hb">לְךָ֛ וּלְבֵיתֶ֑ךָ </span>|<span class="hb">לְךָ֛ </span>|<span class="hb">בֵיתֶ֑ךָ </span>|<span class="hb">יְהוָ֥ה </span>
10|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=25&verse=31&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 25:31" sec="1_Samuel 25:31">1_Samuel 25:31</a>וְלֹ֣א תִהְיֶ֣ה זֹ֣את׀ לְךָ֡ לְפוּקָה֩ וּלְמִכְשֹׁ֨ול לֵ֜ב לַאדֹנִ֗י וְלִשְׁפָּךְ־דָּם֙ חִנָּ֔ם וּלְהֹושִׁ֥יעַ אֲדֹנִ֖י לֹ֑ו וְהֵיטִ֤ב יְהוָה֙ לַֽאדֹנִ֔י וְזָכַרְתָּ֖ אֶת־אֲמָתֶֽךָ׃ ס </span>|<span class="hb">לְךָ֡ לַאדֹנִ֗י </span>|<span class="hb">לְךָ֡ </span>|<span class="hb">אדֹנִ֗י </span>|<span class="hb">לְ</span>
11|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Reges_II&chapter=25&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Kings 25:24" sec="2_Kings 25:24">2_Kings 25:24</a>וַיִּשָּׁבַ֨ע לָהֶ֤ם גְּדַלְיָ֨הוּ֙ וּלְאַנְשֵׁיהֶ֔ם וַיֹּ֣אמֶר לָהֶ֔ם אַל־תִּֽירְא֖וּ מֵעַבְדֵ֣י הַכַּשְׂדִּ֑ים שְׁב֣וּ בָאָ֗רֶץ וְעִבְד֛וּ אֶת־מֶ֥לֶךְ בָּבֶ֖ל וְיִטַ֥ב לָכֶֽם׃ ס </span>|<span class="hb">לָהֶ֤ם וּלְאַנְשֵׁיהֶ֔ם </span>|<span class="hb">לָהֶ֤ם </span>|<span class="hb">אַנְשֵׁיהֶ֔ם </span>|<span class="hb">גְּדַלְיָ֨הוּ֙ </span>
12|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=40&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 40:9" sec="Jeremiah 40:9">Jeremiah 40:9</a>וַיִּשָּׁבַ֨ע לָהֶ֜ם גְּדַלְיָ֨הוּ בֶן־אֲחִיקָ֤ם בֶּן־שָׁפָן֙ וּלְאַנְשֵׁיהֶ֣ם לֵאמֹ֔ר אַל־תִּֽירְא֖וּ מֵעֲבֹ֣וד הַכַּשְׂדִּ֑ים שְׁב֣וּ בָאָ֗רֶץ וְעִבְד֛וּ אֶת־מֶ֥לֶךְ בָּבֶ֖ל וְיִיטַ֥ב לָכֶֽם׃ </span>|<span class="hb">לָהֶ֜ם וּלְאַנְשֵׁיהֶ֣ם </span>|<span class="hb">לָהֶ֜ם </span>|<span class="hb">אַנְשֵׁיהֶ֣ם </span>|<span class="hb">גְּדַלְיָ֨הוּ </span>
13|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Daniel&chapter=9&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Daniel 9:8" sec="Daniel 9:8">Daniel 9:8</a>יְהוָ֗ה לָ֚נוּ בֹּ֣שֶׁת הַפָּנִ֔ים לִמְלָכֵ֥ינוּ לְשָׂרֵ֖ינוּ וְלַאֲבֹתֵ֑ינוּ אֲשֶׁ֥ר חָטָ֖אנוּ לָֽךְ׃ </span>|<span class="hb">לָ֚נוּ לִמְלָכֵ֥ינוּ לְשָׂרֵ֖ינוּ וְלַאֲבֹתֵ֑ינוּ </span>|<span class="hb">לָ֚נוּ </span>|<span class="hb">אֲבֹתֵ֑ינוּ </span>|<span class="hb">בֹּ֣שֶׁת </span>

Let's color the word in the gap differently.

In [8]:
A.show(results, end=3, condensed=False, colorMap={2: 'lightyellow', 3: 'yellow', 5: 'magenta'})



**verse** *1*





**verse** *2*





**verse** *3*



## All gapped phrases

These were particular gaps.
Now we want to get *all* gapped phrases.

We can just lift the special requirement that 
the `preGapWord` has to satisfy a special lexical condition.

In [9]:
query = '''
p:phrase
  wPreGap:word
  wLast:word
  :=

wGap:word
wPreGap <: wGap
wGap < wLast

p || wGap
'''

In [10]:
results = A.search(query)

  2.15s 715 results


Not too bad! We could wait for it. Here are some results.

In [11]:
A.table(results, end=10)

n | phrase | word | word | word
--- | --- | --- | --- | ---
1|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">בֵּ֤ין הַמַּ֨יִם֙ וּבֵ֣ין הַמַּ֔יִם </a></span>|<span class="hb">מַּ֨יִם֙ </span>|<span class="hb">מַּ֔יִם </span>|<span class="hb">אֲשֶׁר֙ </span>
2|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י </a></span>|<span class="hb">עֵ֚שֶׂב </span>|<span class="hb">פְּרִ֞י </span>|<span class="hb">מַזְרִ֣יעַ </span>
3|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12" sec="Genesis 1:12">דֶּ֠שֶׁא עֵ֣שֶׂב וְעֵ֧ץ </a></span>|<span class="hb">עֵ֣שֶׂב </span>|<span class="hb">עֵ֧ץ </span>|<span class="hb">מַזְרִ֤יעַ </span>
4|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:29" sec="Genesis 1:29">אֶת־כָּל־עֵ֣שֶׂב׀ וְאֶת־כָּל־הָעֵ֛ץ </a></span>|<span class="hb">עֵ֣שֶׂב׀ </span>|<span class="hb">עֵ֛ץ </span>|<span class="hb">זֹרֵ֣עַ </span>
5|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25" sec="Genesis 2:25">שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו </a></span>|<span class="hb">שְׁנֵיהֶם֙ </span>|<span class="hb">אִשְׁתֹּ֑ו </span>|<span class="hb">עֲרוּמִּ֔ים </span>
6|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:4" sec="Genesis 4:4">הֶ֨בֶל גַם־ה֛וּא </a></span>|<span class="hb">הֶ֨בֶל </span>|<span class="hb">ה֛וּא </span>|<span class="hb">הֵבִ֥יא </span>
7|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7:8" sec="Genesis 7:8">מִן־הַבְּהֵמָה֙ הַטְּהֹורָ֔ה וּמִן־הַ֨בְּהֵמָ֔ה וּמִ֨ן־הָעֹ֔וף וְכֹ֥ל </a></span>|<span class="hb">בְּהֵמָ֔ה </span>|<span class="hb">כֹ֥ל </span>|<span class="hb">אֲשֶׁ֥ר </span>
8|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7:14" sec="Genesis 7:14">הֵ֜מָּה וְכָל־הַֽחַיָּ֣ה לְמִינָ֗הּ וְכָל־הַבְּהֵמָה֙ לְמִינָ֔הּ וְכָל־הָרֶ֛מֶשׂ לְמִינֵ֑הוּ וְכָל־הָעֹ֣וף לְמִינֵ֔הוּ כֹּ֖ל צִפֹּ֥ור כָּל־כָּנָֽף׃ </a></span>|<span class="hb">רֶ֛מֶשׂ </span>|<span class="hb">כָּנָֽף׃ </span>|<span class="hb">הָ</span>
9|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7:21" sec="Genesis 7:21">כָּל־בָּשָׂ֣ר׀ בָּעֹ֤וף וּבַבְּהֵמָה֙ וּבַ֣חַיָּ֔ה וּבְכָל־הַשֶּׁ֖רֶץ וְכֹ֖ל הָאָדָֽם׃ </a></span>|<span class="hb">בָּשָׂ֣ר׀ </span>|<span class="hb">אָדָֽם׃ </span>|<span class="hb">הָ</span>
10|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7:21" sec="Genesis 7:21">כָּל־בָּשָׂ֣ר׀ בָּעֹ֤וף וּבַבְּהֵמָה֙ וּבַ֣חַיָּ֔ה וּבְכָל־הַשֶּׁ֖רֶץ וְכֹ֖ל הָאָדָֽם׃ </a></span>|<span class="hb">שֶּׁ֖רֶץ </span>|<span class="hb">אָדָֽם׃ </span>|<span class="hb">הַ</span>

If a phrase has multiple gaps, we encounter it multiple times in our results.

We show the two condensed results in Genesis 7:21.

In [12]:
A.show(results, condensed=True, start=9, end=10, colorMap={1: 'lightyellow', 2: 'yellow', 4: 'magenta'})



**verse** *9*





**verse** *10*



If we want just the phrases, and only once, we can run the query in shallow mode, see [advanced](searchAdvanced.ipynb):

In [13]:
gapQueryResults = A.search(query, shallow=True)

  2.25s 671 results


### A different query

We can make an equivalent query to get the gaps.

In [5]:
query = '''
p:phrase
    =: wFirst:word
    wLast:word
    :=

wGap:word
wFirst < wGap
wLast > wGap

p || wGap
'''

Experience has shown that this is a slow query, so we handle it with care.

In [15]:
S.study(query)
S.count(progress=1, limit=8)

   |     0.00s Feature overview: 111 for nodes; 8 for edges; 2 configs; 7 computed
  0.00s Checking search template ...
  0.00s Setting up search space for 4 objects ...
  0.25s Constraining search space with 7 relations ...
  0.26s Setting up retrieval plan ...
  0.30s Ready to deliver results from 1532939 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.00s Counting results per 1 up to 8 ...
   |       26s 1
   |       26s 2
   |       26s 3
   |       26s 4
   |       26s 5
   |       26s 6
   |       47s 7
   |       47s 8
    47s Done: 8 results


This is a good example of a query that is slow to deliver even its first result.
And that is bad, because it is such a straightforward query.

Why is this one so slow, while the previous one went so smoothly?

The crucial thing is the `wGap` word. In the latter template, `wGap` is not embedded in anything.
It is constrained by `wFirst < wGap` and `wGap < wLast`.
However, the way the search strategy works is by examining all possibilities for `wFirst < wGap`
and only then checking whether `wGap < wLast`.
The algorithm cannot check both conditions at the same time.

With embedding relations, things are better. Text-Fabric is heavily optimized to deal with embedding
relationships.

In the former template, we see that the `wGap` is required to be `adjacent` to `wPreGap`, and this one
is embedded in the phrase. Hence there are few cases to consider for `wPreGap`, and per instance
there is only one `wGap`.

> **Lesson**
Try to prevent the use of *free floating* nodes in your template that become constrained
by other spatial relationships than embedding.

### To the rescue
The former template had it right.
Can we rescue the latter template?

We can assume that the phrase and the gap each contain a word in one and the same verse.
Note that phrase and gap may belong to different clauses and sentences.
We assume that a phrase cannot belong to more than two verses, so either the first or the last word
of the phrase is in the same verse as a word in the gap.

In [6]:
query = '''
p:phrase
    =: wFirst:word
    wLast:word
    :=

wGap:word
wFirst < wGap
wLast > wGap

p || wGap

v:verse

v [[ wFirst
v [[ wGap
'''

In [17]:
S.study(query)
S.count(progress=100, limit=3000)

   |     0.00s Feature overview: 111 for nodes; 8 for edges; 2 configs; 7 computed
  0.00s Checking search template ...
  0.00s Setting up search space for 5 objects ...
  0.25s Constraining search space with 9 relations ...
  0.26s Setting up retrieval plan ...
  0.31s Ready to deliver results from 1556152 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.00s Counting results per 100 up to 3000 ...
   |     1.35s 100
   |     2.52s 200
   |     3.32s 300
   |     3.73s 400
   |     4.08s 500
   |     4.90s 600
   |     5.11s 700
   |     6.09s 800
   |     6.41s 900
   |     6.95s 1000
   |     7.53s 1100
   |     7.97s 1200
   |     8.75s 1300
   |       11s 1400
   |       12s 1500
   |       13s 1600
   |       15s 1700
   |       15s 1800
   |       16s 1900
   |       17s 2000
   |       18s 2100
   |       19s 2200
   |       21s 2300
   |       25s 2400
   |       26s 2500
   |       27s 2600
   |       29s 2700
    29s Done: 2707 res

We are going to run this query in `shallow` mode.

In [18]:
results = A.search(query, shallow=True)

    30s 671 results


In [9]:
results = A.search(query, shallow=False)

    30s 2707 results


Shallow mode tends to be quicker, but that does not always materialize.
The number of results agrees with the first query.
Yet we have been lucky, because we required the word in the gap to be in the same verse as the first word in the phrase.
What if we require if it is the last word in the phrase?

In [19]:
query = '''
p:phrase
    =: wFirst:word
    wLast:word
    :=

wGap:word
wFirst < wGap
wLast > wGap

p || wGap

v:verse

v [[ wLast
v [[ wGap
'''

In [20]:
results = A.search(query, shallow=True)

  9.89s 660 results


Then we would not have found all results.

So, this road, although doable, is much less comfortable, performance-wise and logic-wise.

## Check the gaps

In this misty landscape of gaps we need some corroboration that we found the right results.

1. is every node in `gapQueryResults` a phrase?
1. does every phrase in the `gapQueryResults` have a gap?
1. is every gapped phrase contained in `gapQueryResults`?

We check all this by hand coding.

Here is a function that checks whether a phrase has a gap.
If the distance between its end points is greater than the number of words it contains,
it must have a gap.

In [21]:
def hasGap(p):
    words = L.d(p, otype='word')
    return words[-1] - words[0] + 1 > len(words)

Now we can perform the checks.

In [22]:
otypesGood = True
haveGaps = True

for p in gapQueryResults:
    otype = F.otype.v(p)
    if otype != 'phrase':
        print(f'Non phrase detected: {p}) is a {otype}')
        otypesGood = False
        break

    if not hasGap(p):
        print(f'Phrase without a gap: {p}')
        A.pretty(p)
        haveGaps = False
        break

print(f'{len(gapQueryResults)} nodes in query result')
if otypesGood:
    print('1. all nodes are phrases')
if haveGaps:
    print('2. all nodes have gaps')

inResults = True
for p in F.otype.s('phrase'):
    if hasGap(p):
        if p not in gapQueryResults:
            print(f'Gapped phrase outside query results: {p}')
            A.pretty(p)
            inResults = False
            break
            
if inResults:
    print('3. all gapped phrases are contained in the results')

671 nodes in query result
1. all nodes are phrases
2. all nodes have gaps
3. all gapped phrases are contained in the results


Note that by hand coding we can get the gapped phrases much more quickly and securely!

## Custom sets for (non-)gapped phrases

We have obtained a set with all gapped phrases,
and we have paid a price:

* either an expensive query,
* or an inconvenient bit of hand coding.

It would be nice if we could kick-start our queries using this set as a given.
And that is exactly what we are going to do now.

We make two custom sets and give them a name, `gapphrase` for gapped phrases and `conphrase` for non-gapped phrases (consecutive phrases).

In [23]:
customSets = dict(
    gapphrase=gapQueryResults,
    conphrase=set(F.otype.s('phrase')) - gapQueryResults,
)

Suppose we want all verbs that occur in a gapped phrase.

In [24]:
query = '''
gapphrase
  word sp=verb
'''

Note that we have used the foreign name `gapphrase` in our search template, instead of `phrase`.

But we can still run `search()`, provided we tell it what we mean by `gapphrase`. 
We do that by passing the `sets` parameter to `search()`, which should be a dictionary of sets.
Search will look up `gapphrase` in this dictionary, and will use its value, which should be a node set.
That way, it understands that the expression `gapphrase` stands for the nodes in the given node set.

Here we go:

In [25]:
results = A.search(query, sets=customSets)

  0.52s 93 results


In [26]:
A.show(results, start=1, end=3)



**verse** *1*





**verse** *2*





**verse** *3*



That looks good.

We can also apply feature conditions to `gapphrase`:

In [27]:
query = '''
gapphrase function=Subj
'''
results = A.search(query, sets=customSets)
A.show(results, start=1, end=3)

  0.00s 176 results




**verse** *1*





**verse** *2*





**verse** *3*



## Two-phrase clauses

We can find the gaps, but do our minds always reckon with gaps?
Gaps cause unexpected semantics.
Here is a little puzzle.

Suppose we want to count the clauses consisting of exactly two phrases.

Here follows a little journey.
We use a query to find the clauses, check the result with hand-coding, scratch our heads,
refine the query, the hand-coding and our question until we are satisfied.

### Attempt 1

#### By query

The following template should do it:
a clause, starting with a phrase, followed by an adjacent phrase,
which terminates the clause.

In [28]:
query = '''
clause
    =: phrase
    <: phrase
    :=
'''

In [29]:
results = A.search(query)
A.table(results, end=10)

  0.91s 23483 results


n | clause | phrase | phrase
--- | --- | --- | ---
1|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3" sec="Genesis 1:3">יְהִ֣י אֹ֑ור </a></span>|<span class="hb">יְהִ֣י </span>|<span class="hb">אֹ֑ור </span>
2|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">כִּי־טֹ֑וב </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹ֑וב </span>
3|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">אֲשֶׁר֙ מִתַּ֣חַת לָרָקִ֔יעַ </a></span>|<span class="hb">אֲשֶׁר֙ </span>|<span class="hb">מִתַּ֣חַת לָרָקִ֔יעַ </span>
4|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">אֲשֶׁ֖ר מֵעַ֣ל לָרָקִ֑יעַ </a></span>|<span class="hb">אֲשֶׁ֖ר </span>|<span class="hb">מֵעַ֣ל לָרָקִ֑יעַ </span>
5|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=10&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:10" sec="Genesis 1:10">כִּי־טֹֽוב׃ </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹֽוב׃ </span>
6|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">מַזְרִ֣יעַ זֶ֔רַע </a></span>|<span class="hb">מַזְרִ֣יעַ </span>|<span class="hb">זֶ֔רַע </span>
7|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12" sec="Genesis 1:12">כִּי־טֹֽוב׃ </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹֽוב׃ </span>
8|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14" sec="Genesis 1:14">לְהַבְדִּ֕יל בֵּ֥ין הַיֹּ֖ום וּבֵ֣ין הַלָּ֑יְלָה </a></span>|<span class="hb">לְהַבְדִּ֕יל </span>|<span class="hb">בֵּ֥ין הַיֹּ֖ום וּבֵ֣ין הַלָּ֑יְלָה </span>
9|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:15" sec="Genesis 1:15">לְהָאִ֖יר עַל־הָאָ֑רֶץ </a></span>|<span class="hb">לְהָאִ֖יר </span>|<span class="hb">עַל־הָאָ֑רֶץ </span>
10|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:17" sec="Genesis 1:17">לְהָאִ֖יר עַל־הָאָֽרֶץ׃ </a></span>|<span class="hb">לְהָאִ֖יר </span>|<span class="hb">עַל־הָאָֽרֶץ׃ </span>

If we want to have the clauses only, we run it in shallow mode:

In [30]:
clausesByQuery = sorted(A.search(query, shallow=True))

  0.76s 23483 results


#### By hand

Let us check this with a piece of hand-written code.
We want clauses that consist of exactly two phrases.

In [31]:
indent(reset=True)
info('counting ...')

clausesByHand = []
for clause in F.otype.s('clause'):
    phrases = L.d(clause, otype='phrase')
    if len(phrases) == 2:
        clausesByHand.append(clause)
clausesByHand = sorted(clausesByHand)
info(f'Done: found {len(clausesByHand)}')

  0.00s counting ...
  0.62s Done: found 23862


#### The difference

Strange, we end up with more cases. What is happening? Let us compare the results.
We look at the first result where both methods diverge.

We put the difference finding in a little function.

In [32]:
def showDiff(queryResults, handResults):
    diff = [x for x in zip(queryResults, handResults) if x[0] != x[1]]
    if not diff:
        print(f'''
{len(queryResults):>6} queryResults
         are identical with
{len(handResults):>6} handResults
''')
        return
    (rQuery, rHand) = diff[0]
    if rQuery < rHand:
        print(f'clause {rQuery} is a query result but not found by hand')
        toShow = rQuery
    else:
        print(f'clause {rHand} is not a query result but has been found by hand')
        toShow = rHand
    colors = ['aqua', 'aquamarine', 'khaki', 'lavender', 'yellow']
    highlights = {}
    for (i, phrase) in enumerate(L.d(toShow, otype='phrase')):
        highlights[phrase] = colors[i % len(colors)]
        # for atom in L.d(phrase, otype='phrase_atom'):
        #     highlights[atom] = colors[i % len(colors)]
    A.pretty(toShow, withNodes=True, suppress={'lex', 'sp', 'vt', 'vs'}, highlights=highlights)

In [33]:
showDiff(clausesByQuery, clausesByHand)

clause 427931 is not a query result but has been found by hand


Lo and behold:

* the hand-written code is right in a sense: this is a clause that consists exactly of two phrases.
* the query is also right in a sense: the two phrases are not adjacent: there is a gap in the clause between them!

### Attempt 2

#### By hand

We modify the hand-written code such that only clauses qualify if the two phrases are adjacent.

In [34]:
indent(reset=True)
info('counting ...')

clausesByHand2 = []
for clause in F.otype.s('clause'):
    phrases = L.d(clause, otype='phrase')
    if len(phrases) == 2:
        if L.d(phrases[0], otype='word')[-1] + 1 == L.d(phrases[1], otype='word')[0]:
            clausesByHand2.append(clause)
clausesByHand2 = sorted(clausesByHand2)
info(f'Done: found {len(clausesByHand2)}')

  0.00s counting ...
  0.70s Done: found 23399


#### The difference

Now we have less cases. What is going on?

In [35]:
showDiff(clausesByQuery, clausesByHand2)

clause 428692 is a query result but not found by hand


Observe:

This clause has three phrases, but the third one lies inside the second one.

* the hand-written code is right in a sense: this clause has three phrases.
* the query is right in a sense: it contains two adjacent phrases that together span the whole clause.

### Attempt 3

#### By query

Can we adjust the pattern to exclude cases like this? 
Yes, with custom sets, see [advanced](searchAdvanced.ipynb).

Instead of looking through all phrases, we can just consider non gapped phrases only.

Earlier in this notebook we have constructed the set of non-gapped phrases
and put it under the name `conphrase` in the custom sets.

In [36]:
query = '''
clause
    =: conphrase
    <: conphrase
    :=
'''

clausesByQuery2 = sorted(A.search(query, sets=customSets, shallow=True))

  0.75s 23327 results


#### The difference

There is still a difference.

In [37]:
showDiff(clausesByQuery2, clausesByHand2)

clause 428374 is not a query result but has been found by hand


Observe:

This clause has two phrases, the second one has a gap, which coincides with a gap in the clause.

* the hand-written code is right in a sense: this clause has two phrases, adjacent, and they span the whole clause, nothing left out.
* the query is right in a sense: the second phrase is not consecutive.

### Attempt 4

#### By hand

We modify the hand-written code, so that only consecutive clauses qualify.

In [38]:
indent(reset=True)
info('counting ...')

clausesByHand3 = []
for clause in F.otype.s('clause'):
    if hasGap(clause):
        continue
    phrases = L.d(clause, otype='phrase')
    if len(phrases) == 2:
        if L.d(phrases[0], otype='word')[-1] + 1 == L.d(phrases[1], otype='word')[0]:
            clausesByHand3.append(clause)
clausesByHand3 = sorted(clausesByHand3)
info(f'Done: found {len(clausesByHand3)}')

  0.00s counting ...
  0.92s Done: found 23327


#### The difference

Now the number of results agree. But are they really the same?

In [39]:
showDiff(clausesByQuery2, clausesByHand3)


 23327 queryResults
         are identical with
 23327 handResults



### Conclusion

It took four attempts to arrive at the final concept of things that we were looking for.

Sometimes the search template had to be modified, sometimes the hand-written code.

The interplay and systematic comparison between the attempts helped to spot all relevant
configurations of phrases within clauses.

# Spans

Here is another cause of wrong query results: there are sentences that span multiple verses.
Such sentences are not contained in any verse.
That makes that they are easily missed out in queries.

We describe a scenario where that happens.

## Mother clauses

A clause and its mother do not have to be in the same verse.
We are going to fetch are the cases where they are in different verses.

### All mother clauses

But first we fetch all pairs of clauses connected by a mother edge.

In [40]:
query = '''
clause
-mother> clause
'''
allMotherPairs = A.search(query)
A.table(results, end=10)

  0.20s 13907 results


n | clause | phrase | phrase
--- | --- | --- | ---
1|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3" sec="Genesis 1:3">יְהִ֣י אֹ֑ור </a></span>|<span class="hb">יְהִ֣י </span>|<span class="hb">אֹ֑ור </span>
2|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">כִּי־טֹ֑וב </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹ֑וב </span>
3|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">אֲשֶׁר֙ מִתַּ֣חַת לָרָקִ֔יעַ </a></span>|<span class="hb">אֲשֶׁר֙ </span>|<span class="hb">מִתַּ֣חַת לָרָקִ֔יעַ </span>
4|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">אֲשֶׁ֖ר מֵעַ֣ל לָרָקִ֑יעַ </a></span>|<span class="hb">אֲשֶׁ֖ר </span>|<span class="hb">מֵעַ֣ל לָרָקִ֑יעַ </span>
5|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=10&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:10" sec="Genesis 1:10">כִּי־טֹֽוב׃ </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹֽוב׃ </span>
6|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">מַזְרִ֣יעַ זֶ֔רַע </a></span>|<span class="hb">מַזְרִ֣יעַ </span>|<span class="hb">זֶ֔רַע </span>
7|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12" sec="Genesis 1:12">כִּי־טֹֽוב׃ </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹֽוב׃ </span>
8|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14" sec="Genesis 1:14">לְהַבְדִּ֕יל בֵּ֥ין הַיֹּ֖ום וּבֵ֣ין הַלָּ֑יְלָה </a></span>|<span class="hb">לְהַבְדִּ֕יל </span>|<span class="hb">בֵּ֥ין הַיֹּ֖ום וּבֵ֣ין הַלָּ֑יְלָה </span>
9|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:15" sec="Genesis 1:15">לְהָאִ֖יר עַל־הָאָ֑רֶץ </a></span>|<span class="hb">לְהָאִ֖יר </span>|<span class="hb">עַל־הָאָ֑רֶץ </span>
10|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:17" sec="Genesis 1:17">לְהָאִ֖יר עַל־הָאָֽרֶץ׃ </a></span>|<span class="hb">לְהָאִ֖יר </span>|<span class="hb">עַל־הָאָֽרֶץ׃ </span>

### Mother in another verse

Now we modify the query to the effect that mother and daughter must sit in distinct verses.

In [41]:
query = '''
cm:clause
-mother> cd:clause

v1:verse
v2:verse
v1 # v2

cm ]] v1
cd ]] v2
'''
diffMotherPairs = A.search(query)
A.table(results, end=10)

  0.34s 721 results


n | clause | phrase | phrase
--- | --- | --- | ---
1|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3" sec="Genesis 1:3">יְהִ֣י אֹ֑ור </a></span>|<span class="hb">יְהִ֣י </span>|<span class="hb">אֹ֑ור </span>
2|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">כִּי־טֹ֑וב </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹ֑וב </span>
3|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">אֲשֶׁר֙ מִתַּ֣חַת לָרָקִ֔יעַ </a></span>|<span class="hb">אֲשֶׁר֙ </span>|<span class="hb">מִתַּ֣חַת לָרָקִ֔יעַ </span>
4|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">אֲשֶׁ֖ר מֵעַ֣ל לָרָקִ֑יעַ </a></span>|<span class="hb">אֲשֶׁ֖ר </span>|<span class="hb">מֵעַ֣ל לָרָקִ֑יעַ </span>
5|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=10&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:10" sec="Genesis 1:10">כִּי־טֹֽוב׃ </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹֽוב׃ </span>
6|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">מַזְרִ֣יעַ זֶ֔רַע </a></span>|<span class="hb">מַזְרִ֣יעַ </span>|<span class="hb">זֶ֔רַע </span>
7|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12" sec="Genesis 1:12">כִּי־טֹֽוב׃ </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹֽוב׃ </span>
8|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14" sec="Genesis 1:14">לְהַבְדִּ֕יל בֵּ֥ין הַיֹּ֖ום וּבֵ֣ין הַלָּ֑יְלָה </a></span>|<span class="hb">לְהַבְדִּ֕יל </span>|<span class="hb">בֵּ֥ין הַיֹּ֖ום וּבֵ֣ין הַלָּ֑יְלָה </span>
9|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:15" sec="Genesis 1:15">לְהָאִ֖יר עַל־הָאָ֑רֶץ </a></span>|<span class="hb">לְהָאִ֖יר </span>|<span class="hb">עַל־הָאָ֑רֶץ </span>
10|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:17" sec="Genesis 1:17">לְהָאִ֖יר עַל־הָאָֽרֶץ׃ </a></span>|<span class="hb">לְהָאִ֖יר </span>|<span class="hb">עַל־הָאָֽרֶץ׃ </span>

### Mother in same verse

As a check,
we modify the latter query and require `v1` and `v2` to be the same verse, to get the
mother pairs of which both members are in the same verse.

In [42]:
query = '''
cm:clause
-mother> cd:clause

v1:verse
v2:verse
v1 = v2

cm ]] v1
cd ]] v2
'''
sameMotherPairs = A.search(query)
A.table(results, end=10)

  0.27s 13160 results


n | clause | phrase | phrase
--- | --- | --- | ---
1|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3" sec="Genesis 1:3">יְהִ֣י אֹ֑ור </a></span>|<span class="hb">יְהִ֣י </span>|<span class="hb">אֹ֑ור </span>
2|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">כִּי־טֹ֑וב </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹ֑וב </span>
3|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">אֲשֶׁר֙ מִתַּ֣חַת לָרָקִ֔יעַ </a></span>|<span class="hb">אֲשֶׁר֙ </span>|<span class="hb">מִתַּ֣חַת לָרָקִ֔יעַ </span>
4|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">אֲשֶׁ֖ר מֵעַ֣ל לָרָקִ֑יעַ </a></span>|<span class="hb">אֲשֶׁ֖ר </span>|<span class="hb">מֵעַ֣ל לָרָקִ֑יעַ </span>
5|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=10&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:10" sec="Genesis 1:10">כִּי־טֹֽוב׃ </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹֽוב׃ </span>
6|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">מַזְרִ֣יעַ זֶ֔רַע </a></span>|<span class="hb">מַזְרִ֣יעַ </span>|<span class="hb">זֶ֔רַע </span>
7|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12" sec="Genesis 1:12">כִּי־טֹֽוב׃ </a></span>|<span class="hb">כִּי־</span>|<span class="hb">טֹֽוב׃ </span>
8|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14" sec="Genesis 1:14">לְהַבְדִּ֕יל בֵּ֥ין הַיֹּ֖ום וּבֵ֣ין הַלָּ֑יְלָה </a></span>|<span class="hb">לְהַבְדִּ֕יל </span>|<span class="hb">בֵּ֥ין הַיֹּ֖ום וּבֵ֣ין הַלָּ֑יְלָה </span>
9|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:15" sec="Genesis 1:15">לְהָאִ֖יר עַל־הָאָ֑רֶץ </a></span>|<span class="hb">לְהָאִ֖יר </span>|<span class="hb">עַל־הָאָ֑רֶץ </span>
10|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:17" sec="Genesis 1:17">לְהָאִ֖יר עַל־הָאָֽרֶץ׃ </a></span>|<span class="hb">לְהָאִ֖יר </span>|<span class="hb">עַל־הָאָֽרֶץ׃ </span>

### The difference

Let's check if the numbers add up:

* the first query asked for all pairs
* the second query asked for pairs with members in different verses
* the third query asked for pairs with members in the same verse

Then the results of the second and third query combined should
equal the results of the first query.

That makes sense.

Still, let's check:

In [43]:
discrepancy = len(allMotherPairs) - len(diffMotherPairs) - len(sameMotherPairs)
print(discrepancy)

26


The numbers do not add up. We are missing cases. Why?

Clauses may cross verse boundaries. In that case they are not part of a verse, and hence our latter two queries
do not detect them. Let's count how many verse boundary crossing clauses there are.

In [44]:
query = '''
clause
    =: first:word
    last:word
    :=
v1:verse
    w1:word
v2:verse
    w2:word
    
first = w1
last = w2
v1 # v2
'''
results = A.search(query)

  2.05s 50 results


Some of these verse spanning clauses do not have mothers or are not mothers. Let's count the cases where two clauses
are in a mother relation and at least one of them spans a verse.

We need two queries for that. These queries are almost similar. One retrieves the clause pairs where the mother
crosses verse boundaries, and the other where the daughter does so.

But we are programmers. We do not have to repeat ourselves:

In [45]:
queryCommon = '''
c1:clause
-mother> c2:clause

c3:clause
    =: first:word
    last:word
    :=
v1:verse
    w1:word
v2:verse
    w2:word
    
first = w1
last = w2
v1 # v2
'''

query1 = f'''
{queryCommon}
c1 = c3
'''
query2 = f'''
{queryCommon}
c2 = c3
'''

results1 = A.search(query1, silent=True)
results2 = A.search(query2, silent=True)
spannersByQuery = {(r[0], r[1]) for r in results1 + results2}
print(f'{len(spannersByQuery):>3} spanners are missing')
print(f'{discrepancy:>3} missing cases were detected before')
print(f'{discrepancy - len(spannersByQuery):>3} is the resulting disagreement')

 26 spanners are missing
 26 missing cases were detected before
  0 is the resulting disagreement


We may find the mother clause pairs in which it least one member is verse spanning by hand-coding in an easier way:

Starting with the set of all mother pairs, we filter out any pair that has a verse spanner.

In [46]:
spannersByHand = set()

for (c1, c2) in allMotherPairs:
    if not (
        L.u(c1, otype='verse')
        and
        L.u(c2, otype='verse')
    ):
        spannersByHand.add((c1, c2))
        
len(spannersByHand)

26

And, to be completely sure:

In [47]:
spannersByHand == spannersByQuery

True

### By custom sets

If we are content with the clauses that do not span verses,
we can put them in a set, and modify the queries by replacing `clause` by `conclause`
and bind the right set to it.

Here we go. In one cell we run the queries to get all pairs, the mother-daughter-in-separate-verses pairs,
and the mother-daughter-in-same-verses pair and we do the math of checking.

In [48]:
conClauses = {c for c in F.otype.s('clause') if L.u(c, otype='verse')}
customSets = dict(conclause=conClauses)

print('All pairs')
allPairs = A.search('''
conclause
-mother> conclause
''', 
    sets=customSets,
)

print('Different verse pairs')
diffPairs = A.search('''
cm:conclause
-mother> cd:conclause

v1:verse
v2:verse
v1 # v2

cm ]] v1
cd ]] v2
''',
    sets=customSets,
)

print('Same verse pairs')
samePairs = A.search('''
cm:conclause
-mother> cd:conclause

v1:verse
v2:verse
v1 = v2

cm ]] v1
cd ]] v2
''',
    sets=customSets,
)

allPairSet = set(allPairs)
diffPairSet = {(r[0], r[1]) for r in diffPairs}
samePairSet = {(r[0], r[1]) for r in samePairs}

print(f'Intersection same-verse/different-verse pairs: {samePairSet & diffPairSet}')
print(f'All pairs is union of same-verse/different-verse pairs: {allPairSet == (samePairSet | diffPairSet)}')

All pairs
  0.23s 13881 results
Different verse pairs
  0.21s 721 results
Same verse pairs
  0.34s 13160 results
Intersection same-verse/different-verse pairs: set()
All pairs is union of same-verse/different-verse pairs: True


## Lessons

* mix programming with composing queries;
* a good way to do so is custom sets;
* use programming for processing results;
* find the balance between queries and hand-coding.

# Next

**You have now finished the search tutorial.**

If you are interested in reproducing MQL queries in Text-Fabric search templates,
see [fromMQL](searchFromMQL.ipynb).

---

[basic](search.ipynb)
[advanced](searchAdvanced.ipynb)
[sets](searchSets.ipynb)
[relations](searchRelations.ipynb)
[quantifiers](searchQuantifiers.ipynb)
[rough](searchRough.ipynb)
gaps